In [14]:
# Парсер объявлений на сайте auto.ru по тестовому заданию на стажера тех отдела TrainingData.Solutions

In [15]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from tqdm import tqdm
import requests

from selenium.common import TimeoutException

In [16]:
options = Options()
# options.add_argument('--headless')
# options.add_argument("--disable-infobars")
# options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
# options.add_argument('--blink-settings=imagesEnabled=false')
webdriver = webdriver.Chrome(options=options)
webdriver.get(f'https://auto.ru/')

"""Обходим капчу и всплывающие окна"""
if webdriver.find_elements(By.XPATH, '//*[@id="confirm-button"]'):
    WebDriverWait(webdriver, 15).until(ec.element_to_be_clickable((By.XPATH, '//*[@id="confirm-button"]'))).click()
if webdriver.find_elements(By.XPATH, '/html/body/div[1]/div/div/form/div[3]/div/div[1]/div[1]/input'):
    WebDriverWait(webdriver, 15).until(ec.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/form/div[3]/div/div[1]/div[1]/input'))).click()
WebDriverWait(webdriver, 20).until(ec.presence_of_element_located((By.XPATH, '/html/body/div[3]/div/div/div[2]')))
if webdriver.find_elements(By.XPATH, '/html/body/div[3]/div/div/div[2]'):
    WebDriverWait(webdriver, 15).until(ec.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div/div/div[2]'))).click()

"""Создаем список из ссылок 10ти первых марок машин на сайте"""
urls = []
j = 2
k = 0
for i in range(1, 11):
    car = webdriver.find_element(By.XPATH, f'//*[@id="LayoutIndex"]/div/div/div[1]/div[1]/div[2]/div[2]/div/div[{str(j)}]/a[{str(i - k)}]')
    car_url = car.get_attribute('href')
    urls.append(car_url)
    if i >= 7:
        j = 3
        k = 7

"""Создаем директорию для хранения ссылок и сохраняем туда веб-страницы"""
for i in range(len(urls)):
    webdriver.get(urls[i])
    car_brand = urls[i].split('/')[4]
    if webdriver.find_elements(By.XPATH, '/html/body/div[1]/div/div/form/div[3]/div/div[1]/div[1]/input'):
        WebDriverWait(webdriver, 5).until(ec.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/form/div[3]/div/div[1]/div[1]/input'))).click()
    if webdriver.find_elements(By.XPATH, '/html/body/div[2]/div/div/div[2]'):
        WebDriverWait(webdriver, 5).until(ec.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div/div[2]'))).click()
    webdriver.execute_script("window.scrollTo(0, 1080)")

    soup = BeautifulSoup(webdriver.page_source, 'html.parser')
    car_brand_links_directory = 'brand_links/'
    if not os.path.exists(car_brand_links_directory):
        os.makedirs(car_brand_links_directory)
    with open(f'{car_brand_links_directory}{car_brand}.html', 'w', encoding="utf-8") as file:
        file.write(str(soup))
print(urls)
webdriver.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.71)
Stacktrace:
	GetHandleVerifier [0x00007FF7FDCC4D02+56194]
	(No symbol) [0x00007FF7FDC304B2]
	(No symbol) [0x00007FF7FDAD76AA]
	(No symbol) [0x00007FF7FDAB0AFD]
	(No symbol) [0x00007FF7FDB4C9AB]
	(No symbol) [0x00007FF7FDB6201F]
	(No symbol) [0x00007FF7FDB45C23]
	(No symbol) [0x00007FF7FDB14A45]
	(No symbol) [0x00007FF7FDB15AD4]
	GetHandleVerifier [0x00007FF7FE03D5BB+3695675]
	GetHandleVerifier [0x00007FF7FE096197+4059159]
	GetHandleVerifier [0x00007FF7FE08DF63+4025827]
	GetHandleVerifier [0x00007FF7FDD5F029+687785]
	(No symbol) [0x00007FF7FDC3B508]
	(No symbol) [0x00007FF7FDC37564]
	(No symbol) [0x00007FF7FDC376E9]
	(No symbol) [0x00007FF7FDC28094]
	BaseThreadInitThunk [0x00007FFC6200257D+29]
	RtlUserThreadStart [0x00007FFC6336AA58+40]


In [ ]:
"""Отбираем по 10 первых моделей машин каждой марки"""
car_links = []
links_limit = 10
brand_directory = 'brand_links'
if not os.path.exists(brand_directory):
    os.makedirs(brand_directory)
for file in os.listdir(brand_directory):
    soup = BeautifulSoup(open(f'{brand_directory}/{file}', 'r', encoding="utf-8").read(), 'html.parser')
    counter = 0
    for div_class in soup.find_all('div', class_='ListingItem__main'):
        try:
            car_link = div_class.find('a', class_='Link ListingItemTitle__link').get('href')
            car_links.append(car_link)
            counter += 1
            if counter >= links_limit:
                break
        except AttributeError:
            continue

In [ ]:
from selenium import webdriver

options = Options()
# options.add_argument('--headless')
# options.add_argument("--disable-infobars")
# options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
# options.add_argument('--blink-settings=imagesEnabled=false')
webdriver = webdriver.Chrome(options=options)

"""Создаем директории марка машины/модель и загружаем туда до 5ти фото из объявления"""
for i in tqdm(range(len(car_links))):
    try:
        webdriver.get(car_links[i])
        car_brand = car_links[i].split('/')[6]

        if webdriver.find_elements(By.XPATH, '//*[@id="confirm-button"]'):
            WebDriverWait(webdriver, 15).until(ec.element_to_be_clickable((By.XPATH, '//*[@id="confirm-button"]'))).click()
        if webdriver.find_elements(By.XPATH, '/html/body/div[1]/div/div/form/div[3]/div/div[1]/div[1]/input'):
            WebDriverWait(webdriver, 15).until(ec.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div/form/div[3]/div/div[1]/div[1]/input'))).click()
        WebDriverWait(webdriver, 20).until(ec.presence_of_element_located((By.XPATH, '/html/body/div[3]/div/div/div[2]')))
        if webdriver.find_elements(By.XPATH, '/html/body/div[3]/div/div/div[2]'):
            WebDriverWait(webdriver, 15).until(ec.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div/div/div[2]'))).click()

        soup = BeautifulSoup(webdriver.page_source, 'html.parser')
        title = soup.find('h1', class_='CardHead__title').text

        if webdriver.find_element(By.XPATH, '//*[@id="app"]/div/div[2]/div[3]/div/div[2]/div/div[2]/div/div[2]/div[5]/div[2]/div/div[2]/div[1]/div[4]/div/img'):
            WebDriverWait(webdriver, 5).until(ec.element_to_be_clickable((By.XPATH, '//*[@id="app"]/div/div[2]/div[3]/div/div[2]/div/div[2]/div/div[2]/div[5]/div[2]/div/div[2]/div[1]/div[4]/div/img'))).click()

        images = webdriver.find_elements(By.CLASS_NAME, 'ImageGalleryFullscreenVertical__image')

        image_count = 0
        image_limit = 5
        for index, image in enumerate(images):
            image_url = image.get_attribute('src')
            if image_url:
                response = requests.get(image_url, stream=True)
                result_directory = f'result/{car_brand}/{title}_{i}/'  # Добавляем i из-за наличия дубликатов в объявлениях
                if not os.path.exists(result_directory):
                    os.makedirs(result_directory)
                image_path = os.path.join(result_directory, f'{title}_{index + 1}.jpg')

                # Сохраняем изображение
                with open(image_path, 'wb') as file:
                    for chunk in response.iter_content(chunk_size=128):
                        file.write(chunk)
                image_count += 1

                if image_count >= image_limit:
                    break
    except TimeoutException:
        continue

webdriver.quit()